In [1]:
import tensorflow as tf

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
from autograd import numpy as np

In [3]:
df_train = pd.read_csv("./traindata.csv", dtype=np.uint8)

In [4]:
df_train.head(3)

,id,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [5]:
df_train.shape

(60000, 786)

In [6]:
df_imgs = df_train.drop(['label', 'id'], axis=1)
x_original = df_imgs.as_matrix().T

print("x_original shape: ", x_original.shape)

x_original shape:  (784, 60000)


In [7]:
# extract edge-based features
import data_transformer
x_original_edgebased_features = data_transformer.edge_transformer(x_original)   

print('shape of original input ', x_original.shape)
print('shape of transformed input ', x_original_edgebased_features.shape)

shape of original input  (784, 60000)
shape of transformed input  (1352, 60000)


In [8]:
y_original = df_train['label'].as_matrix()

print("y_original shape: ", y_original.shape)

y_original shape:  (60000,)


In [12]:
num_sample = 5000
inds = np.random.permutation(y_original.shape[0])[:num_sample]
x_sample = x_original_edgebased_features[:,inds].T
y_sample = y_original[inds]

In [13]:
print("x_sample shape: ", x_sample.shape)
print("y_sample shape: ", y_sample.shape)

x_sample shape:  (5000, 1352)
y_sample shape:  (5000,)


In [14]:
x = x_sample
y = y_sample

In [15]:
xmax = np.max(x)
x_scale = x/xmax

In [16]:
input_dim = x_scale.shape[1]
nb_classes = y.shape[0]

print("input_dim: ", input_dim)
print("nb_classes: ", nb_classes)

input_dim:  1352
nb_classes:  5000


In [17]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [18]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [19]:
model.fit(x_scale, y, epochs=5, verbose=2)

Epoch 1/5
 - 3s - loss: 0.3796 - acc: 0.8804
Epoch 2/5
 - 2s - loss: 0.1337 - acc: 0.9592
Epoch 3/5
 - 2s - loss: 0.0888 - acc: 0.9712
Epoch 4/5
 - 2s - loss: 0.0563 - acc: 0.9814
Epoch 5/5
 - 2s - loss: 0.0359 - acc: 0.9896


In [20]:
df_testdata = pd.read_csv("./testdata.csv", dtype=np.uint8)
df_testdata.head(3)

,id,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
df_testimgs = df_testdata.drop(['label', 'id'], axis=1)
xtest = df_testimgs.as_matrix().T

print("xtest shape:", xtest.shape)

xtest shape: (784, 10000)


In [29]:
xtest_edgebased_features = data_transformer.edge_transformer(xtest)   

print('shape of original input ', xtest_edgebased_features.shape)

shape of original input  (1352, 10000)


In [30]:
ytest = df_testdata['label'].as_matrix()
# ytest = ytest.reshape((10000, 1)).T
print("ytest shape:", ytest.shape)

ytest shape: (10000,)


In [31]:
xtest_scale = xtest_edgebased_features/xmax

xtest_scale = xtest_scale.T

In [32]:
model.evaluate(xtest_scale, ytest)

10000/10000 [==============================] - 1s 59us/step


[0.09810038092313335, 0.9706]

In [33]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(input_dim, 
                                input_dim=input_dim, 
                                activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(nb_classes, activation=tf.nn.softmax))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [34]:
model.fit(x_scale, y, epochs=10, verbose=2)

Epoch 1/10
 - 21s - loss: 0.6099 - acc: 0.8476
Epoch 2/10
 - 22s - loss: 0.1637 - acc: 0.9494
Epoch 3/10
 - 20s - loss: 0.0827 - acc: 0.9756
Epoch 4/10
 - 20s - loss: 0.0609 - acc: 0.9806
Epoch 5/10
 - 20s - loss: 0.0448 - acc: 0.9862
Epoch 6/10
 - 20s - loss: 0.0259 - acc: 0.9932
Epoch 7/10
 - 19s - loss: 0.0143 - acc: 0.9966
Epoch 8/10
 - 18s - loss: 0.0067 - acc: 0.9992
Epoch 9/10
 - 18s - loss: 0.0050 - acc: 0.9994
Epoch 10/10
 - 18s - loss: 0.0031 - acc: 1.0000


In [35]:
model.evaluate(xtest_scale, ytest)

10000/10000 [==============================] - 5s 460us/step


[0.09663175543043763, 0.973]

aaa

In [135]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(input_dim, 
                                input_dim=input_dim, 
                                activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(input_dim,
                                activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(input_dim,
                                activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(nb_classes, activation=tf.nn.softmax))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [136]:
model.fit(x_scale, y, epochs=20, verbose=2)

Epoch 1/10
 - 16s - loss: 0.8755 - acc: 0.7748
Epoch 2/10
 - 13s - loss: 0.2174 - acc: 0.9352
Epoch 3/10
 - 13s - loss: 0.1445 - acc: 0.9548
Epoch 4/10
 - 13s - loss: 0.0902 - acc: 0.9726
Epoch 5/10
 - 13s - loss: 0.0692 - acc: 0.9776
Epoch 6/10
 - 13s - loss: 0.0419 - acc: 0.9868
Epoch 7/10
 - 13s - loss: 0.0312 - acc: 0.9898
Epoch 8/10
 - 13s - loss: 0.0558 - acc: 0.9834
Epoch 9/10
 - 13s - loss: 0.0421 - acc: 0.9890
Epoch 10/10
 - 14s - loss: 0.0468 - acc: 0.9880


In [137]:
model.evaluate(xtest_scale, ytest)

10000/10000 [==============================] - 5s 477us/step


[0.27374397583268584, 0.9426]